## Problema de clasificación de estado de Cirrosis usando PandasAI

In [17]:
import pandas as pd
from pandasai import SmartDataframe
from dotenv import dotenv_values
import os
# Environment key
keys = dotenv_values("../.env-token")
os.environ["PANDASAI_API_KEY"] = keys["PANDASAI_API_KEY"]

Voy a utilizar el LLM default --> BambooLM para poder hacer una comparación con AIDE 

In [18]:
file = pd.read_csv('Data/liver_cirrhosis.csv')
file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   N_Days         25000 non-null  int64  
 1   Status         25000 non-null  object 
 2   Drug           25000 non-null  object 
 3   Age            25000 non-null  int64  
 4   Sex            25000 non-null  object 
 5   Ascites        25000 non-null  object 
 6   Hepatomegaly   25000 non-null  object 
 7   Spiders        25000 non-null  object 
 8   Edema          25000 non-null  object 
 9   Bilirubin      25000 non-null  float64
 10  Cholesterol    25000 non-null  float64
 11  Albumin        25000 non-null  float64
 12  Copper         25000 non-null  float64
 13  Alk_Phos       25000 non-null  float64
 14  SGOT           25000 non-null  float64
 15  Tryglicerides  25000 non-null  float64
 16  Platelets      25000 non-null  float64
 17  Prothrombin    25000 non-null  float64
 18  Stage 

In [19]:
texto = "Make a classifier for the 3 values in Stage Column"

In [20]:
smartDataframe = SmartDataframe(file, description="An isolated dataset with cirrhosis patients data")

In [21]:
response  = smartDataframe.chat("Make a classifier for the 3 values in Stage Column")
response

Traceback (most recent call last):
  File "/Users/alejandro/Documents/Repositorios/alejandromadrid/.python-pandasai-venv/lib/python3.10/site-packages/pandasai/pipelines/chat/generate_chat_pipeline.py", line 310, in run
    ).run(input)
  File "/Users/alejandro/Documents/Repositorios/alejandromadrid/.python-pandasai-venv/lib/python3.10/site-packages/pandasai/pipelines/pipeline.py", line 137, in run
    raise e
  File "/Users/alejandro/Documents/Repositorios/alejandromadrid/.python-pandasai-venv/lib/python3.10/site-packages/pandasai/pipelines/pipeline.py", line 101, in run
    step_output = logic.execute(
  File "/Users/alejandro/Documents/Repositorios/alejandromadrid/.python-pandasai-venv/lib/python3.10/site-packages/pandasai/pipelines/chat/code_execution.py", line 113, in execute
    raise e
  File "/Users/alejandro/Documents/Repositorios/alejandromadrid/.python-pandasai-venv/lib/python3.10/site-packages/pandasai/pipelines/chat/code_execution.py", line 85, in execute
    result = self.

"Unfortunately, I was not able to answer your question, because of the following error:\n\ncould not convert string to float: 'C'\n"

El código ha tardado en ejecutarse <label style="color:red"> </label>

Sacamos el código recién ejecutado

In [23]:
print(smartDataframe.last_code_executed)

df = pd.concat(dfs)
df['Stage'] = df['Stage'].map({'1': 1, '2': 2, '3': 3})
X = df.drop(columns=['Stage'])
y = df['Stage']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
result = {'type': 'string', 'value': f'The accuracy of the classifier for the Stage column is {accuracy}.'}


Probamos el código recién ejecutado

In [28]:
# ponermos los imports que no ofrece la solución de PandaAI
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [29]:

df = file
df['Stage'] = df['Stage'].map({'1': 1, '2': 2, '3': 3})
X = df.drop(columns=['Stage'])
y = df['Stage']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
result = {'type': 'string', 'value': f'The accuracy of the classifier for the Stage column is {accuracy}.'}

ValueError: could not convert string to float: 'C'

### Como solucionar el error

Como no hemos conseguido que haga un encoder y luego haga el entrenamiento de modelo. vamos a pedirle que primero haga un encoder de las variables categóricas o un preprocesamiento a mano.

In [32]:
file.head()

,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,2221,C,Placebo,18499,F,N,Y,N,N,0.5,149.0,4.04,227.0,598.0,52.70,57.0,256.0,9.9,0
1,1230,C,Placebo,19724,M,Y,N,Y,N,0.5,219.0,3.93,22.0,663.0,45.00,75.0,220.0,10.8,0
2,4184,C,Placebo,11839,F,N,N,N,N,0.5,320.0,3.54,51.0,1243.0,122.45,80.0,225.0,10.0,0
3,2090,D,Placebo,16467,F,N,N,N,N,0.7,255.0,3.74,23.0,1024.0,77.50,58.0,151.0,10.2,0
4,2105,D,Placebo,21699,F,N,Y,N,N,1.9,486.0,3.54,74.0,1052.0,108.50,109.0,151.0,11.5,0


Como podemos observar, tenemos muchas columnas con el tipo objeto que no podemos parsear a float, es por eso que PandasAI no es capaz de hacer lo  suyo, para ayudarle vamos a aplicar una serie de cambios para que este pueda trabajar bien.

Vamos a hacer un análisis básico para ver cuantos valores tenemos que intercambiar

In [37]:
file['Drug'].unique()

array(['Placebo', 'D-penicillamine'], dtype=object)